                                        ##############################
                                        #   **Cities Fingerprint**   #
                                        #     Shortest Path Data     #                                                  
                                        #                            #
                                        #   		  codename: k1   #
                                        #                            #
                                        #   version = 201604251112   #
                                        #    Issues known: none      #
                                        #                            #
                                        #   A script by AndresPDLR   #
                                        ##############################


    This program performs statistical analysis over city simulation made
    by Module_SHORTEST_PATHS

<div class="alert alert-success">
**Requirements**

</div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline
plt.style.use('ggplot')

import statsmodels.api as sm
import math as mt
import datetime
import ast
from sys import argv
import csv
import os
from simpledbf import Dbf5
import scipy
import statistics as st

from localidades_100k import localidades_100k_dictionary
from localidades import localidades_dictionary

In [2]:
poblacion_localidades_100k = sum(localidades_100k_dictionary.values())
poblacion_urbana = sum(localidades_dictionary.values())

print(poblacion_localidades_100k / poblacion_urbana)

0.6169473816526283


<div class="alert alert-success">
**Functions**
</div>

In [3]:
def de_string_list(list_as_string):
    ''' Transform string representation of list to list '''
    if type(list_as_string) != list:
        try:
            list_as_list = ast.literal_eval(list_as_string)
            return list_as_list
        except ValueError:
            return list_as_string
    else:
        return list_as_string

In [4]:
def beta_city(localidad):
    '''
    >> Performs distance analysis over a city landscape
    
    Input:
        - localidad
    
    Output:
        - plot
        - csv with average distances
        - beta & R2
    '''
    try:
        df_simulation = pd.read_csv('{}_simulation_results.csv'.format(localidad), header=None, names=['G_FID', 'Distances'])
        df_simulation = df_simulation.dropna()
        df_simulation['G_FID'] = df_simulation['G_FID'].apply(de_string_list)
        df_simulation['Distances'] = df_simulation['Distances'].apply(de_string_list)
        g_fids = []
        for x in df_simulation['G_FID']:
            if x[0] not in g_fids:
                g_fids.append(x[0])
            if x[1] not in g_fids:
                g_fids.append(x[1])
        averages = {}
        for g_fid in g_fids:
            rv = []
            for row in df_simulation['G_FID']:
                if g_fid in row:
                    temporary_df = df_simulation.loc[df_simulation['G_FID'] == row]
                    d_distance = temporary_df['Distances'].tolist()[0][1]
                    rv.append(d_distance)
            averages[g_fid] = sum(rv) / len(rv)
        writer = csv.writer(open('C:/Users/Andres/{}_average_distance.csv'.format(localidad), 'wt'))
        for key, value in averages.items():
            writer.writerow([str(key), value])
        X = []
        Y = []
        for row in df_simulation['Distances']:
            x = row[0]
            y = row[1]
            X.append(x)
            Y.append(y)
        OLS_results = sm.OLS(Y,X).fit()
        beta = OLS_results.params.tolist()[0]
        R2 = OLS_results.rsquared
        xmax = max(X)
        ymax = max(Y)
        Y_sd = st.stdev(Y)
        # Plot
        '''
        plt.figure(figsize=(18,12))
        beta_label = '$\mathrm{β}$'
        plt.title("Simulation for {} >> {} = {} with R2 = {}\n".format(localidad, beta_label, round(beta, 5), round(R2, 5)), fontsize = 20)
        plt.xlabel("Euclidean Distance")
        plt.ylabel("Dijkstra Distance")        
        plt.scatter(X, Y, s = 10, c = 'b', alpha=0.7)
        plt.plot(X, X, '-')
        X_beta = []
        for x in X:
            X_beta.append(x*beta)
        plt.plot(X, X_beta, '-')
        plt.xlim((0, xmax))
        plt.ylim((0, ymax))
        plt.savefig('{}_simulation.png'.format(localidad), dpi = 350)
        plt.show()
        '''
        return [beta, R2, Y_sd]
    except ValueError:
        pass

In [5]:
beta_dictionary = {}

for localidad in localidades_100k_dictionary.keys():
    try:
        beta_phi = beta_city(localidad)
        beta = beta_phi[0]
        sd = beta_phi[2]
        beta_dictionary[str(localidad)] = (beta, sd)
    except TypeError:
        pass

In [6]:
localidades = []
betas = []
standard_deviations = []
for k, v in beta_dictionary.items():
    localidades.append(k)
    betas.append(v[0])
    standard_deviations.append(v[1])

beta_pd = pd.DataFrame({'Localidades': localidades, 'Beta': betas, 'SD': standard_deviations})
beta_pd.head()

,Beta,Localidades,SD
0,1.788059,270040001,0.032916
1,1.258961,240280001,0.062208
2,1.221583,150330001,0.042531
3,1.273466,150570001,0.028086
4,1.280020,110200001,0.050041


In [7]:
beta_pd.to_csv("C:\\Users\\Andres\\betas.csv", index=False)